# Методы отбора признаков

## Задание 1

Вам предстоит создать модель для предсказания цен на недвижимость в Калифорнии. Однако перед этим необходимо провести отбор признаков для улучшения качества модели.

Используйте два метода отбора признаков, **SBS (Sequential Backward Selection)** и **SBFS (Sequential Backward Floating Selector)**, на датасете с данными о недвижимости в Калифорнии.

После отбора признаков объедините отобранные признаки и обучите модель на оптимальном наборе признаков. Ваша модель должна обучаться не менее, чем на **6 признаках**. В качестве метрики оптимизации используйте `neg_mean_squared_error`.

In [ ]:
# Импорт необходимых библиотек
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Загрузка датасета с данными о недвижимости в Калифорнии
california_housing = fetch_california_housing()
X = california_housing.data
y = california_housing.target

print(f"Размер датасета: {X.shape}")
print(f"Названия признаков: {california_housing.feature_names}")
print(f"\nОписание датасета:")
print(california_housing.DESCR[:500])

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nРазмер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")

In [ ]:
print("="*70)
print("ОТБОР ПРИЗНАКОВ")
print("="*70)

# Инициализация модели
lr = LinearRegression()

# Инициализация Sequential Feature Selector для backward selection
print("\n1️⃣ Sequential Backward Selection (SBS)...")
sbs = SequentialFeatureSelector(
    lr,
    k_features=6,  # минимум 6 признаков
    forward=False,  # backward selection
    floating=False,  # без floating
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1
)

# Обучение SBS на обучающем наборе
sbs = sbs.fit(X_train, y_train)

# Выбор признаков с помощью SBS - создаём преобразованные данные
selected_features_sbs = sbs.transform(X_train)
print(f"Выбранные признаки SBS (индексы): {list(sbs.k_feature_idx_)}")
print(f"Выбранные признаки SBS (названия): {[california_housing.feature_names[i] for i in sbs.k_feature_idx_]}")
print(f"Лучший score SBS: {sbs.k_score_:.4f}")
print(f"Размерность selected_features_sbs: {selected_features_sbs.shape}")

# Инициализация Sequential Feature Selector с включённым параметром floating для SBFS
print("\n2️⃣ Sequential Backward Floating Selection (SBFS)...")
sbfs = SequentialFeatureSelector(
    lr,
    k_features=6,  # минимум 6 признаков
    forward=False,  # backward selection
    floating=True,  # с floating
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1
)

# Обучение SBFS на обучающем наборе
sbfs = sbfs.fit(X_train, y_train)

# Выбор признаков с помощью SBFS - создаём преобразованные данные
selected_features_sbfs = sbfs.transform(X_train)
print(f"Выбранные признаки SBFS (индексы): {list(sbfs.k_feature_idx_)}")
print(f"Выбранные признаки SBFS (названия): {[california_housing.feature_names[i] for i in sbfs.k_feature_idx_]}")
print(f"Лучший score SBFS: {sbfs.k_score_:.4f}")
print(f"Размерность selected_features_sbfs: {selected_features_sbfs.shape}")

# Объединение индексов выбранных признаков из SBS и SBFS для обучения
selected_features_indices = list(set(list(sbs.k_feature_idx_) + list(sbfs.k_feature_idx_)))
selected_features_indices.sort()  # сортируем для удобства

print(f"\n3️⃣ Объединение признаков:")
print(f"Объединённые признаки (индексы): {selected_features_indices}")
print(f"Объединённые признаки (названия): {[california_housing.feature_names[i] for i in selected_features_indices]}")
print(f"Общее количество признаков: {len(selected_features_indices)}")

# Создаём переменную selected_features для обучения модели
selected_features = selected_features_indices
X_train_selected = X_train[:, selected_features]
X_test_selected = X_test[:, selected_features]

print(f"\nРазмерность X_train_selected: {X_train_selected.shape}")
print(f"Размерность X_test_selected: {X_test_selected.shape}")

In [ ]:
print("="*70)
print("ОБУЧЕНИЕ МОДЕЛИ НА ОТОБРАННЫХ ПРИЗНАКАХ")
print("="*70)

# Обучение модели линейной регрессии на выбранных признаках
model = LinearRegression()
model.fit(X_train_selected, y_train)

# Предсказания
y_train_pred = model.predict(X_train_selected)
y_test_pred = model.predict(X_test_selected)

# Метрики
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"\n📊 Метрики на обучающей выборке:")
print(f"  MSE: {train_mse:.4f}")
print(f"  RMSE: {train_rmse:.4f}")
print(f"  R²: {train_r2:.4f}")

print(f"\n📊 Метрики на тестовой выборке:")
print(f"  MSE: {test_mse:.4f}")
print(f"  RMSE: {test_rmse:.4f}")
print(f"  R²: {test_r2:.4f}")

print(f"\n✅ Модель успешно обучена на {len(selected_features)} признаках!")

In [ ]:
print("="*70)
print("СРАВНЕНИЕ С МОДЕЛЬЮ НА ВСЕХ ПРИЗНАКАХ")
print("="*70)

# Обучение модели на всех признаках для сравнения
model_all = LinearRegression()
model_all.fit(X_train, y_train)

y_test_pred_all = model_all.predict(X_test)
test_mse_all = mean_squared_error(y_test, y_test_pred_all)
test_rmse_all = np.sqrt(test_mse_all)
test_r2_all = r2_score(y_test, y_test_pred_all)

print(f"\n📊 Модель на ВСЕХ признаках ({X.shape[1]} признаков):")
print(f"  MSE: {test_mse_all:.4f}")
print(f"  RMSE: {test_rmse_all:.4f}")
print(f"  R²: {test_r2_all:.4f}")

print(f"\n📊 Модель на ОТОБРАННЫХ признаках ({len(selected_features)} признаков):")
print(f"  MSE: {test_mse:.4f}")
print(f"  RMSE: {test_rmse:.4f}")
print(f"  R²: {test_r2:.4f}")

print(f"\n📈 Изменение метрик:")
print(f"  ΔMSE: {test_mse - test_mse_all:.4f} ({((test_mse - test_mse_all) / test_mse_all * 100):.2f}%)")
print(f"  ΔR²: {test_r2 - test_r2_all:.4f} ({((test_r2 - test_r2_all) / test_r2_all * 100):.2f}%)")
print(f"  Уменьшение признаков: {X.shape[1] - len(selected_features)} ({((X.shape[1] - len(selected_features)) / X.shape[1] * 100):.1f}%)")

## Итоги

В этом задании мы:

1. **Загрузили датасет** California Housing с 8 признаками
2. **Применили SBS** (Sequential Backward Selection) для отбора 6 лучших признаков
3. **Применили SBFS** (Sequential Backward Floating Selection) для отбора 6 лучших признаков
4. **Объединили результаты** обоих методов
5. **Обучили модель** LinearRegression на отобранных признаках
6. **Сравнили результаты** с моделью на всех признаках

### Преимущества отбора признаков:
- Уменьшение размерности данных
- Потенциальное улучшение обобщающей способности модели
- Снижение вычислительных затрат
- Улучшение интерпретируемости модели